<a href="https://colab.research.google.com/github/cytoscape/cytoscape-automation/blob/master/for-scripters/Python/Visual-property-mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

While creating visual property maps is straight forward, choosing the name of the virtual property is not. It would be helpful to have a cookbook item showing how to build and use a visual property map, and then identify the meaning of each visual property name. As an educated Cytoscape user, it took me a couple of hours to get this right … a cookbook could make this an economical proposition.

Recommendation: Add cookbook example showing how to create and use a visual property map

Recommendation: Add cookbook article listing visual property names and what they apply to

# Introduction to Mapping Styles to Attributes

Harsh Sharma

2025-01-17

**Cytoscape is an open source software platform for integrating, visualizing, and analyzing measurement data in the context of networks.**

This tutorial intends ....


In [2]:
import IPython
import py4cytoscape as p4c

In [3]:
p4c.cytoscape_version_info()

{'apiVersion': 'v1',
 'cytoscapeVersion': '3.10.3',
 'automationAPIVersion': '1.11.0',
 'py4cytoscapeVersion': '1.11.0'}

## Understanding Styling in Cytoscape

Styles are divided into three main categories:

1. **Default**  
   Applies a uniform style to the entire network without exceptions.

2. **Mapping**  
   Dynamically applies styles to nodes and edges based on their properties (e.g., size based on degree, color based on a specific attribute).

3. **Bypass**  
   Directly overrides other styles for *selected* nodes or edges, providing targeted customization.

---

### Focus of This Tutorial: Mapping

In this tutorial, we’ll focus on **Mapping**, which allows you to create data-driven visualizations. You'll learn how to map node and edge attributes to various visual properties like color, size, and transparency. This approach makes your networks more insightful and visually compelling. 

## Introduction to Style Options

Before diving into this tutorial, it’s helpful to have a basic understanding of the styling options available in Cytoscape. If you’re new to these concepts or need a refresher, we recommend reviewing the [*Styles* section in the Cytoscape manual](https://manual.cytoscape.org/en/stable/Styles.html#).

One of the best things about working with Cytoscape through py4cytoscape is that you don't need to rely solely on the graphical user interface to explore and interact with styles. You can retrieve all the style options programmatically, which makes the process more efficient and flexible. Whether you’re analyzing a large network or automating your workflow, this approach is invaluable.

However, one important note: while you can retrieve style options through code, identifying the shapes associated with specific names may still require referring to the Styles tab—unless you’re already familiar with them.


In [3]:
p4c.get_arrow_shapes() # List of all available arrow shapes.

['CROSS_DELTA',
 'HALF_TOP',
 'OPEN_CIRCLE',
 'DELTA',
 'OPEN_HALF_CIRCLE',
 'HALF_CIRCLE',
 'T',
 'CIRCLE',
 'DIAMOND_SHORT_1',
 'OPEN_SQUARE',
 'DIAMOND_SHORT_2',
 'DELTA_SHORT_2',
 'OPEN_DIAMOND',
 'DELTA_SHORT_1',
 'OPEN_DELTA',
 'SQUARE',
 'ARROW',
 'NONE',
 'HALF_BOTTOM',
 'CROSS_OPEN_DELTA',
 'DIAMOND',
 'ARROW_SHORT']

In [4]:
p4c.get_line_styles() # List of all available line styles

['ZIGZAG',
 'BACKWARD_SLASH',
 'FORWARD_SLASH',
 'EQUAL_DASH',
 'PARALLEL_LINES',
 'SINEWAVE',
 'DASH_DOT',
 'CONTIGUOUS_ARROW',
 'SEPARATE_ARROW',
 'SOLID',
 'VERTICAL_SLASH',
 'LONG_DASH',
 'MARQUEE_DASH',
 'DOT',
 'MARQUEE_DASH_DOT',
 'MARQUEE_EQUAL']

In [5]:
p4c.get_node_shapes() # List of all available node shapes

['VEE',
 'DIAMOND',
 'TRIANGLE',
 'RECTANGLE',
 'PARALLELOGRAM',
 'OCTAGON',
 'HEXAGON',
 'ROUND_RECTANGLE',
 'ELLIPSE']

In [10]:
p4c.get_visual_property_names() # Names of all visual properties.

['COMPOUND_NODE_PADDING',
 'COMPOUND_NODE_SHAPE',
 'DING_RENDERING_ENGINE_ROOT',
 'EDGE',
 'EDGE_BEND',
 'EDGE_CURVED',
 'EDGE_LABEL',
 'EDGE_LABEL_AUTOROTATE',
 'EDGE_LABEL_BACKGROUND_COLOR',
 'EDGE_LABEL_BACKGROUND_SHAPE',
 'EDGE_LABEL_BACKGROUND_TRANSPARENCY',
 'EDGE_LABEL_COLOR',
 'EDGE_LABEL_FONT_FACE',
 'EDGE_LABEL_FONT_SIZE',
 'EDGE_LABEL_POSITION',
 'EDGE_LABEL_ROTATION',
 'EDGE_LABEL_TRANSPARENCY',
 'EDGE_LABEL_WIDTH',
 'EDGE_LINE_TYPE',
 'EDGE_PAINT',
 'EDGE_SELECTED',
 'EDGE_SELECTED_PAINT',
 'EDGE_SOURCE_ARROW_SELECTED_PAINT',
 'EDGE_SOURCE_ARROW_SHAPE',
 'EDGE_SOURCE_ARROW_SIZE',
 'EDGE_SOURCE_ARROW_UNSELECTED_PAINT',
 'EDGE_STACKING',
 'EDGE_STACKING_DENSITY',
 'EDGE_STROKE_SELECTED_PAINT',
 'EDGE_STROKE_UNSELECTED_PAINT',
 'EDGE_TARGET_ARROW_SELECTED_PAINT',
 'EDGE_TARGET_ARROW_SHAPE',
 'EDGE_TARGET_ARROW_SIZE',
 'EDGE_TARGET_ARROW_UNSELECTED_PAINT',
 'EDGE_TOOLTIP',
 'EDGE_TRANSPARENCY',
 'EDGE_UNSELECTED_PAINT',
 'EDGE_VISIBLE',
 'EDGE_WIDTH',
 'EDGE_Z_ORDER',
 'NETWOR

### Filtering Visual Properties

When working with Cytoscape styles, you might want to filter visual properties based on specific keywords, such as "location" This can help you quickly identify relevant properties for customization. Let's explore how to do this!

#### Step 1: Filtering with a List Comprehension

You can use a simple list comprehension to filter visual properties that contain the word "location" (case-insensitive). Here's how:

In [29]:
[i for i in p4c.get_visual_property_names() if "location" in i.lower()]

['NETWORK_CENTER_X_LOCATION',
 'NETWORK_CENTER_Y_LOCATION',
 'NETWORK_CENTER_Z_LOCATION',
 'NODE_X_LOCATION',
 'NODE_Y_LOCATION',
 'NODE_Z_LOCATION']

#### Step 2: Creating a Reusable Function

If you need to filter visual properties by various keywords repeatedly, it's helpful to define a reusable function. Here's an example:

In [8]:
# Define a function to filter visual properties by a given keyword
def filter_properties(keyword : str):
    """
    Filter visual properties containing the specified keyword.

    Parameters:
        keyword (str): The keyword to search for in visual property names.

    Returns:
        list: A list of visual property names containing the keyword (case-insensitive).
    """
    return [i for i in p4c.get_visual_property_names() if keyword.lower() in i.lower()]

# Use the function to filter properties containing "location"
filter_properties("location")

['NETWORK_CENTER_X_LOCATION',
 'NETWORK_CENTER_Y_LOCATION',
 'NETWORK_CENTER_Z_LOCATION',
 'NODE_X_LOCATION',
 'NODE_Y_LOCATION',
 'NODE_Z_LOCATION']

#### Try It Yourself!

Experiment with different keywords, like `"color"`, `"transparency"`, or `"size"`. 

This simple function and list comprehension make it easy to explore and filter the extensive list of visual properties in Cytoscape.

## Loading Session
Let's open a Cytoscape demo Session file:

In [35]:
p4c.open_session()

Opening sampleData/sessions/Yeast Perturbation.cys...


{}

## Starting with Defaults: Exploring Visual Property Defaults

Before diving into property mappings, let's start by exploring **default styles**. These represent the base styling applied to all nodes and edges in your network unless overridden by mappings or bypasses.

#### Retrieving Default Visual Properties

You can programmatically retrieve the default values of visual properties for a given style. For example, let’s check the default **node shape** in the "default" style:

In [34]:
# Retrieve the default node shape in the "default" style
p4c.get_visual_property_default('NODE_SHAPE', style_name='default')

'ELLIPSE'

#### What Does This Do?

The function **get_visual_property_default()** retrieves the current **default value** of a specified visual property for a given style. In this case:
- **'NODE_SHAPE'** specifies the visual property we are querying.
- **style_name='default'** ensures we are looking at the defaults for the "default" style.

---

#### Try It Yourself!
- Experiment with other visual properties, such as 'EDGE_WIDTH', 'NODE_FILL_COLOR', or 'NETWORK_BACKGROUND_PAINT'.  

### Customizing Default Visual Properties in Cytoscape

There are two approaches: using **specific functions** and the **general function** for setting defaults.

---

#### Specific Functions: Quick and Easy Customization

For most use cases, specific functions can be used to update default visual properties. These functions follow a consistent naming pattern:

```python
p4c.set_<property>_default(value, style_name='style')
```

- **`<property>`**: The visual property you want to customize (e.g., `node_shape`, `node_color`, etc.).  
- **`value`**: The desired default value (e.g., `'ELLIPSE'`, `50`, or a color like `'#D3D3D3'`).  
- **`style_name`**: (Optional) The name of the style to modify. Defaults to `'default'` if not specified.

For a full list of specific functions, refer to the [**Styles -> Style Defaults**](https://py4cytoscape.readthedocs.io/en/latest/reference/styles.html#module-py4cytoscape.style_defaults) section of the `py4cytoscape` documentation.

---

### General Function: Flexible Customization for All Properties

If a specific function isn’t available for a property, you can use the general-purpose `set_visual_property_default()` function. This function allows you to define the property and its value as a dictionary.

```python
p4c.set_visual_property_default(style_string, style_name=None)
```

- **`style_string`**: A dictionary specifying the visual property and its value. Example:  
  ```python
  {'visualProperty': 'NODE_SIZE', 'value': '35'}
  ```  
- **`style_name`**: (Optional) The name of the style to modify. Defaults to `'default'`.


In [32]:
# Set all nodes to have an Rectangle shape using the specific function
p4c.set_node_shape_default('RECTANGLE', style_name='default')

''

In [33]:
# Set the default node shape to 'ELLIPSE' using the general function
style_string = {'visualProperty': 'NODE_SHAPE', 'value': 'ELLIPSE'}
p4c.set_visual_property_default(style_string, style_name='default')

''

#### Key Functions for Default Styling

Here’s a quick overview of commonly used functions:  
- **set_node_shape_default()**: Sets the default shape for nodes (e.g., 'ELLIPSE', 'RECTANGLE').  
- **set_node_size_default()**: Adjusts the default size of nodes.  
- **set_node_color_default()**: Changes the default color of nodes (using hex codes, e.g., '#D3D3D3').  
- **set_node_border_width_default()**: Sets the width of the node border.  
- **set_node_font_size_default()**: Adjusts the default font size for node labels.  

Again, a complete list can be found in the [*Styles -> Style Defaults*](https://py4cytoscape.readthedocs.io/en/latest/reference/styles.html#module-py4cytoscape.style_defaults) section of the docs. 

These functions allow you to programmatically define the look and feel of your network. Before applying advanced mappings or customizations to make your network more dynamic

In [5]:
# p4c.set_visual_style('default')
# p4c.set_node_shape_default('ELLIPSE', style_name='default')
# p4c.lock_node_dimensions(True, style_name='default')
# p4c.set_node_size_default(50, style_name='default')
# p4c.set_node_color_default('#D3D3D3', style_name='default')
# p4c.set_node_border_width_default(2, style_name='default')
# p4c.set_node_color_default('#616060', style_name='default')
# p4c.set_node_font_size_default(14, style_name='default')

''

## Mapping Styles to Attributes